# Generate Layout 

The first step is to create a layout and test manifest for your project, so that you can relate measurements and analyses back to your source layout.

You can generate the layout with any layout tool. In this notebook you can use kfactory.

## Generate layout

Kfactory allows you to write metadata directly into the cell

You can download this [generate_layout.py](./generate_layout.py) file

In [ ]:
import kfactory as kf
from generate_layout import top
import csv

c = top()
c.write_gds("test_chip.gds")
c.plot()

## Generate cell table

In your sample GDS, you have the device settings annotated on the GDS.
Here you can read them from the GDS and write a cell table, which we can use to associate measurement data with the devices on the GDS.
However, you can use any method you prefer to generate your table.

In [ ]:
from gdsfactory.labels import get_test_manifest

ct = get_test_manifest(c, one_setting_per_column=True)
ct.to_csv("cell_table.csv", index=False)
print(ct)

## Generate device table

For generating the device table you need to use Kfactory.

A device needs the X, Y position on the layout.
Note that you can instantiate a Cell many times into a die.

In [ ]:
import kfactory as kf
import csv

kf.kcl.read("test_chip.gds")
c = kf.kcl["top"]
sweep = c.kcl["cutback"]
csvpath = "device_table.csv"

analysis = "[cutback]"
analysis_parameters = "[{}]"
length = 20

with open(csvpath, "w") as f:
    writer = csv.writer(f)
    writer.writerow(
        [
            "cell",
            "x",
            "y",
            # "analysis",
            # "analysis_parameters",
        ]
    )
    iterator = sweep._kdb_cell.begin_instances_rec()
    iterator.targets = "cutback_component_add_fiber_array*"
    while not iterator.at_end():
        _c = c.kcl[iterator.inst_cell().cell_index()]
        name = _c.name
        _disp = (iterator.trans() * iterator.inst_trans()).disp
        params = name.split("-")  # we extract the width from the device name
        writer.writerow(
            [
                name,
                _disp.x,
                _disp.y,
                # analysis,
                # analysis_parameters,
            ]
        )
        iterator.next()

You can take a look a the contents of the device manifest you created.

In [ ]:
import pandas as pd

dt = pd.read_csv(csvpath)
dt

## Merge device and cell table

In [ ]:
ct.set_index("cell", inplace=True)
dt.set_index("cell", inplace=True)
df = pd.merge(dt, ct, left_on="cell", right_index=True)
df

In [ ]:
df.columns

In [ ]:
df.to_csv("device_table.csv")